<a href="https://colab.research.google.com/github/sreebalineni/qgss-2024/blob/main/QGSS2024Lib1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit[visualization]==1.1.0
# Use the following if you are on MacOS/zsh
#!pip install 'qiskit[visualization]'==1.1.0
!pip install qiskit_aer
!pip install qiskit_ibm_runtime
!pip install matplotlib
!pip install pylatexenc
!pip install prototype-zne
!pip install git+https://github.com/qiskit-community/Quantum-Challenge-Grader.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 10.7 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=80c0fdf5587adf12ccf3a6e8ab22443d977e57f8f6fbba14181a3c15d2414de9
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
%set_env QXToken=491e2a8cc2f920ff4d51d4d2929a2a0c1ea69029b860a19064d4b9bf387c0cffb63ca6fcc336651b0badef51de15bd286695b3b48f322838b75898e491eed3ff

env: QXToken=491e2a8cc2f920ff4d51d4d2929a2a0c1ea69029b860a19064d4b9bf387c0cffb63ca6fcc336651b0badef51de15bd286695b3b48f322838b75898e491eed3ff


In [3]:
# import of required libraries and modules
from qc_grader.challenges.qgss_2024 import *

from math import pi
from qiskit.circuit.library import QFT
from qiskit.providers.fake_provider import GenericBackendV2, generic_backend_v2
generic_backend_v2._NOISE_DEFAULTS["cx"] = (5.99988e-06, 6.99988e-06, 1e-5, 5e-3)

from qiskit import transpile, QuantumCircuit
from qiskit.circuit import Gate
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import CouplingMap, StagedPassManager, PassManager, AnalysisPass, TransformationPass
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.common import generate_unroll_3q, generate_embed_passmanager
from qiskit.quantum_info import hellinger_fidelity
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.dagcircuit import DAGCircuit
from qiskit_ibm_runtime.fake_provider import FakeTorino

# Transpiler Passes
## Layout passes
from qiskit.transpiler.passes.layout.csp_layout import CSPLayout
from qiskit.transpiler.passes.layout.dense_layout import DenseLayout
from qiskit.transpiler.passes.layout.sabre_layout import SabreLayout
from qiskit.transpiler.passes.layout.vf2_layout import VF2Layout
from qiskit.transpiler.passes.layout.trivial_layout import TrivialLayout

## Routing passes
from qiskit.transpiler.passes.routing.basic_swap import BasicSwap
from qiskit.transpiler.passes.routing.lookahead_swap import LookaheadSwap
from qiskit.transpiler.passes.routing.sabre_swap import SabreSwap
from qiskit.transpiler.passes.routing.stochastic_swap import StochasticSwap
from qiskit.transpiler.passes.routing.star_prerouting import StarPreRouting

## Synthesis passes (passes for the translation stage)
from qiskit.circuit import SessionEquivalenceLibrary
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary
from qiskit.transpiler.passes.basis.basis_translator import BasisTranslator
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HighLevelSynthesis
### The next pass could also be considered an optimization pass.
from qiskit.transpiler.passes.synthesis.unitary_synthesis import UnitarySynthesis

## Optimization passes
from qiskit.transpiler.passes.optimization.collect_1q_runs import Collect1qRuns
from qiskit.transpiler.passes.optimization.collect_2q_blocks import Collect2qBlocks
from qiskit.transpiler.passes.optimization.consolidate_blocks import ConsolidateBlocks
from qiskit.transpiler.passes.optimization.commutative_cancellation import CommutativeCancellation

In [4]:
# get an abstract quantum circuit from Qiskit's library of circuits
num_qubits = 10
qc = QFT(num_qubits, do_swaps=False)
qc.draw()

┌──────┐
q_0: ┤0     ├
     │      │
q_1: ┤1     ├
     │      │
q_2: ┤2     ├
     │      │
q_3: ┤3     ├
     │      │
q_4: ┤4     ├
     │  QFT │
q_5: ┤5     ├
     │      │
q_6: ┤6     ├
     │      │
q_7: ┤7     ├
     │      │
q_8: ┤8     ├
     │      │
q_9: ┤9     ├
     └──────┘

In [45]:
#Hint: see https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit
def get_qc_characteristics(qc):
    # Your work goes here!
    # determine the quantum circuit depth of `qc` and assign it to `depth`
    depth = qc.depth()
    # determine the number of qubits in `qc` and assign it to `num_qubits`
    num_qubits = qc.num_qubits
    # determine the operations in `qc` and assign it to `ops`
    ops =qc.count_ops()
    # determine the number of n-qubit operations (with n larger than 1) in `qc` and assign it to `num_multi_qubit_ops`,
    num_multi_qubit_ops = len([op for op in qc.data if op.operation.num_qubits > 1])
    # do not modify the next line
    return {"depth":depth, "num_qubits":num_qubits, "ops":ops, "num_multi_qubit_ops": num_multi_qubit_ops}

In [46]:
# print quantum circuit characteristics
def print_qc_characteristics(qc):
    characteristics = get_qc_characteristics(qc)
    print("Quantum circuit characteristics")
    print("  Depth:", characteristics['depth'])
    print("  Number of qubits:", characteristics['num_qubits'])
    print("  Operations:", dict(characteristics['ops']))
    print("  Number of multi-qubit Operations:", characteristics['num_multi_qubit_ops'])

print_qc_characteristics(qc)

Quantum circuit characteristics
  Depth: 1
  Number of qubits: 10
  Operations: {'QFT': 1}
  Number of multi-qubit Operations: 1


In [47]:
grade_lab1_ex1(get_qc_characteristics)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [48]:
qc_dec = qc.decompose()
get_qc_characteristics(qc_dec)
qc_dec.draw(fold=-1)

┌───┐
q_0: ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────────────────────────────■──────────────────────────────────────■──────────────────────────────────■───────────────────────────■───────────────────────■──────────────────■─────────────■────────■───────┤ H ├
                                                                                                                                                                                                                                                                     │                                            │                                      │                                  │                           │                       │                  │       ┌───┐ │        │P(π/2) └───┘
q_1: ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────────■──────────┼──────────────────────────────────■─────────┼────────────────────────────■─────────┼────────────────────────■─────────┼──────────────────■────────┼──────────────■────────┼─────────■────────┼───────┤ H ├─┼────────■────────────
                                                                                                                                                                                                                   │                                      │          │                                  │         │                            │         │                        │         │                  │        │        ┌───┐ │        │         │P(π/2)  │       └───┘ │P(π/4)               
q_2: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────■─────────┼────────────────────────────■─────────┼──────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼───────────────■────────┼─────────┼─────────■────────┼────────┼────────┤ H ├─┼────────┼─────────■────────┼─────────────■─────────────────────
                                                                                                                                                                      │                                  │         │                            │         │          │                        │         │         │                   │        │         │         ┌───┐ │        │         │         │P(π/2)  │        │        └───┘ │P(π/4)  │                  │P(π/8)                             
q_3: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────■─────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼──────────┼───────────────■────────┼─────────┼─────────┼──────────■────────┼────────┼─────────┼─────────┤ H ├─┼────────┼─────────┼─────────■────────┼────────┼──────────────■────────┼──────────────────■───────────────────────────────────
                                                                                                                                │                           │         │                        │         │         │                   │        │         │          │         ┌───┐ │        │         │         │          │P(π/2)  │        │         │         └───┘ │P(π/4)  │         │                  │P(π/8)  │                       │P(π/16)                                               
q_4: ─────────────────────────────────────────────────────────────────────────────────────

In [49]:
backend = GenericBackendV2(num_qubits)
print("Supported basis gates:", backend.operation_names)


Supported basis gates: ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure']


In [50]:
# transpile the qft quantum circuit for the basis gate of our example backend
qc_synth = generate_preset_pass_manager(2, backend=backend).run(qc)
qc_synth.draw(fold=-1)

global phase: 0
                                                                                                                                                               ┌───┐    ┌───────────┐┌───┐┌──────────┐                      ┌───┐   ┌───────────┐   ┌───┐    ┌──────────┐                           ┌───┐ ┌──────────┐                                ┌───┐┌─────────┐                  ┌───┐ ┌──────────┐                    ┌───┐┌──────────┐    ┌────┐   ┌────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
q_5 -> 0 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├────┤ Rz(-π/32) ├┤ X ├┤ Rz(π/32) ├──────────────────────┤ X ├───┤ Rz(-π/16) ├───┤ X ├────┤ Rz(π/16) ├───────────────────────────┤ X ├─┤ Rz(-π/8) ├────────────────────────────────┤ X ├┤ Rz(π/8) ├──────────────────┤ X ├─┤ Rz(-π/4) ├────────────────────┤ X ├┤ Rz(3π/4) ├────┤ √X ├───┤ Rz(3.0925) ├───────────────────────────────────────────────────────────────────■────────────────────────────────────■────────────────────────────────■────────────────────────────────────────■────────■─────────────────────■──────────────────────────────────────■─────────────────────────────────────────■───────■──────────────────────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                               └─┬─┘    └───────────┘└─┬─┘└──────────┘                      └─┬─┘   └───────────┘   └─┬─┘    └──────────┘                           └─┬─┘ └──────────┘             ┌───┐┌────────────┐└─┬─┘└─────────┘┌───┐┌───────────┐└─┬─┘ └──────────┘ ┌───┐┌────────────┐└─┬─┘└──────────┘    └────┘   └───┬───┬────┘ ┌───────────┐                        ┌───┐    ┌───────────┐       │           ┌───┐ ┌──────────┐       │            ┌───┐┌───────────┐  │                   ┌───┐ ┌──────────┐   │      ┌─┴─┐    ┌───────────┐┌─┴─┐┌──────────┐                        │                ┌───┐      ┌──────────┐  │       │      ┌───┐ ┌─────────┐       │      ┌───┐┌──────────┐            ┌───┐ ┌──────────┐   ┌────┐   ┌──────────┐                                                                                                                                             
q_2 -> 1 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼─────────────────────┼──────────────────────────────────────┼───────────────────────┼───────────────────────────────────────────────┼────────────────────────────┤ X ├┤ Rz(-π/256) ├──┼─────────────┤ X ├┤ Rz(π/256) ├──┼────────────────┤ X ├┤ Rz(-π/128) ├──┼───────────────────────────────┤ X ├──────┤ Rz(π/128) ├────────────────────────┤ X ├────┤ Rz(-π/64) ├───────┼───────────┤ X ├─┤ Rz(π/64) ├───────┼────────────┤ X ├┤ Rz(-π/32) ├──┼───────────────────┤ X ├─┤ Rz(π/32) ├───┼──────┤ X ├────┤ Rz(-π/16) ├┤ X ├┤ Rz(π/16) ├────────────────────────┼────────────────┤ X ├──────┤ Rz(-π/8) ├──┼───────┼──────┤ X ├─┤ Rz(π/8) ├───────┼──────┤ X ├┤ Rz(-π/4) ├────────────┤ X ├─┤ Rz(3π/4) ├───┤ √X ├───┤ Rz(7π/8) ├──────────────────■──────────────────────■────────────────■─────────────────■─────────────────────

In [51]:
print_qc_characteristics(qc_synth)


Quantum circuit characteristics
  Depth: 65
  Number of qubits: 10
  Operations: {'rz': 101, 'cx': 90, 'sx': 10}
  Number of multi-qubit Operations: 90


In [66]:
# Your work goes here!
# assign a 10-qubit linear `CouplingMap` to the variable cm
cm = CouplingMap(couplinglist=[[i ,i+1] for i in range(9)])

# add your transpiled quantum circuit to the next line
qc_routed = transpile(qc, backend=backend, coupling_map=cm)
qc_routed.draw(fold=-1)

global phase: 0.001534
         ┌─────────┐┌────┐┌─────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            ┌──────────┐   ┌────┐  ┌─────────────┐   ┌────┐  ┌──────────┐           ┌─────────┐   ┌────┐   ┌─────────────┐   ┌────┐   ┌─────────┐                                                                                                          ┌──────────┐   ┌────┐  ┌─────────────┐   ┌────┐  ┌──────────┐     ┌─────────┐┌────┐┌─────────────┐┌────┐┌─────────┐                                                                                                               ┌──────────┐   ┌────┐   ┌───────────┐    ┌────┐   ┌──────────┐              ┌─────────┐   ┌────┐  ┌───────────┐   ┌────┐   ┌─────────┐                                                                                                                                 ┌──────────┐   ┌────┐  ┌───────────┐   ┌────┐  ┌──────────┐             ┌─────────┐    ┌────┐    ┌───────────┐    ┌────┐    ┌─────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   ┌─────────┐ ┌────┐┌─────────┐      ┌─────────┐┌────┐┌─────────┐      ┌─────────┐ ┌────┐ ┌─────────┐                                              ┌──────────┐┌────┐┌───────────┐┌────┐┌──────────┐     ┌─────────┐ ┌────┐┌───────────┐┌────┐┌─────────┐                                                                                               ┌──────────┐ ┌────┐┌───────────┐ ┌────┐┌──────────┐     ┌─────────┐┌────┐┌───────────┐┌────┐┌──────────┐                                                                                                                                                                              ┌─────────┐                                                                                                                                                                                                                                                                                                                                 ┌──────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
q_5 -> 0 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [67]:
# grade your work!
grade_lab1_ex2(qc_routed)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [69]:
pm_staged = StagedPassManager()
# replace the n-qubit QFT operation with its decomposition in two-qubit gates
pm_staged.init = generate_unroll_3q(None)
# initialize the layout stage with an empty pass manager
pm_staged.layout = PassManager()
# set a 'trivial' initial layout, i.e. each qubit in the quantum circuit with index i
# is mapped to the physical qubit on a device with the same index
pm_staged.layout += TrivialLayout(cm)

# do not modify the next line
pm_staged.layout += generate_embed_passmanager(cm)

In [73]:
# Your work goes here!

# See the first cells in this notebook or
#  https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/routing for potential routing passes
pm_staged.routing = PassManager(BasicSwap(cm))
qc_routed = pm_staged.run(qc)
print_qc_characteristics(qc_routed)
qc_routed.draw(fold=-1)

Quantum circuit characteristics
  Depth: 99
  Number of qubits: 10
  Operations: {'swap': 240, 'cp': 45, 'h': 10}
  Number of multi-qubit Operations: 285


┌───┐
q_0 -> 0 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────────────■─────────────■─────────────■────────────■────────────■────────────■───────────■───────────■───────┤ H ├
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                ┌───┐    │P(π/512)     │P(π/256)     │P(π/128)     │P(π/64)     │P(π/32)     │P(π/16)     │P(π/8)     │P(π/4)     │P(π/2) └───┘
q_1 -> 1 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────────────■─────────────■────────────■────────────■────────────■───────────■───────────■───────┤ H ├─X──■──────────X──■──────────X──■──────────X──■─────────X──■─────────X──■─────────X──■────────X──■────────X──■────────────
                                                                                                                                                                                                                                                                                                                                                                                                    ┌───┐    │P(π/256)     │P(π/128)     │P(π/64)     │P(π/32)     │P(π/16)     │P(π/8)     │P(π/4)     │P(π/2) └───┘ │             │             │             │            │            │            │           │           │               
q_2 -> 2 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────────────■────────────■────────────■────────────■───────────■───────────■───────┤ H ├─X──■──────────X──■──────────X──■─────────X──■─────────X──■─────────X──■────────X──■────────X──■─────────X───X──────X──────X──────X──────X──────X──────X─────X──────X─────X──────X─────X──────X─────X─────X─────X─────X───────────────
                                                                                                                                                                                                                                                                                                      ┌───┐    │P(π/128)     │P(π/64)     │P(π/32)     │P(π/16)     │P(π/8)     │P(π/4)     │P(π/2) └───┘ │             │             │            │            │            │           │           │            │          │             │             │            │            │            │            │           │                     
q_3 -> 3 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────■────────────■

In [74]:
grade_lab1_ex3(pm_staged)


Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [75]:
# Your work goes here!

# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/synthesis for potential translation passes
pm_staged.translation = PassManager(BasisTranslator(equivalence_library=SessionEquivalenceLibrary, target_basis=backend.operation_names))
qc_routed_synth = pm_staged.run(qc)
print_qc_characteristics(qc_routed_synth)
qc_routed_synth.draw(fold=-1)

Quantum circuit characteristics
  Depth: 351
  Number of qubits: 10
  Operations: {'cx': 810, 'rz': 155, 'sx': 10}
  Number of multi-qubit Operations: 810


global phase: 1.5723
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    ┌───┐┌─────────────┐┌───┐┌────────────┐                       ┌───┐┌────────────┐┌───┐┌───────────┐                       ┌───┐┌────────────┐┌───┐┌───────────┐                       ┌───┐┌────────────┐┌───┐┌───────────┐                      ┌───┐┌───────────┐┌───┐┌──────────┐                      ┌───┐┌───────────┐┌───┐┌──────────┐                      ┌───┐┌───────────┐┌───┐┌──────────┐                     ┌───┐┌──────────┐┌───┐┌─────────┐                     ┌───┐┌──────────┐┌───┐┌─────────┐┌─────────┐┌────┐┌─────────┐
q_0 -> 0 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [76]:
# grade your work!
grade_lab1_ex4(pm_staged)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [85]:
qk_qc = generate_preset_pass_manager(2,coupling_map=cm, basis_gates=backend.operation_names).run(qc)
print_qc_characteristics(qk_qc)

Quantum circuit characteristics
  Depth: 238
  Number of qubits: 10
  Operations: {'rz': 313, 'sx': 186, 'cx': 176, 'x': 40}
  Number of multi-qubit Operations: 176


In [86]:
def noisy_sim(qc, backend):
    # We add measurement operations to the input quantum circuit and then run it on the specified backend
    # A GenericBackendV2 automatically constructs a default model of the expected noise processes,
    # so backend.run would return noisy simulation results
    return backend.run(qc.measure_all(inplace=False), shots=7*1024).result().get_counts()

own_transpiler_sim = noisy_sim(qc_routed_synth, backend)
qiskit_transpiler_sim = noisy_sim(qk_qc, backend)
reference_sim =  noisy_sim(transpile(qc.decompose(), backend=backend), BasicSimulator())

print("Own transpiler fidelity", round(hellinger_fidelity(own_transpiler_sim, reference_sim), 4))
print("Qiskit transpiler fidelity", round(hellinger_fidelity(qiskit_transpiler_sim, reference_sim), 4))

Own transpiler fidelity 0.0482
Qiskit transpiler fidelity 0.9179


In [96]:
# Your work goes here!

pm_opt = StagedPassManager()
# Select pass managers and passes for these five stages as you see suitable, note: not every stage must contain a pass.
# Passes are documented at https://docs.quantum.ibm.com/api/qiskit/transpiler_passes

# Replace the n-qubit QFT operation with its decomposition in two-qubit gates
pm_opt.init = generate_unroll_3q(None)
# Initialize the layout stage with an empty pass manager
pm_opt.layout = PassManager()
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/layout for potential layouting passes
pm_opt.layout += TrivialLayout(cm)
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/routing for potential routing passes
pm_opt.routing = PassManager(StarPreRouting())
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/synthesis for potential translation passes
pm_opt.translation =  PassManager([Collect2qBlocks(),ConsolidateBlocks(basis_gates=backend.operation_names),UnitarySynthesis(basis_gates=backend.operation_names)])
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/optimization for potential optimization passes
pm_opt.optimization = None
# scheduling is not needed in this lab - we will not run the quantum circuit on a real device
# pm_opt.scheduling = None

# do not modify the next line
pm_opt.layout += generate_embed_passmanager(cm)

qc_opt = pm_opt.run(qc)

In [97]:
grade_lab1_ex5(pm_opt)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [98]:
print_qc_characteristics(qc_opt)
reduction_ratio = round(100-100*(get_qc_characteristics(qc_opt)['num_multi_qubit_ops']/get_qc_characteristics(qk_qc)['num_multi_qubit_ops']), 3)
print("Reduction in two-qubit gates compared to qiskit {}%!".format(reduction_ratio))
qc_opt.draw(fold=-1)

Quantum circuit characteristics
  Depth: 183
  Number of qubits: 10
  Operations: {'rz': 397, 'sx': 186, 'cx': 133, 'x': 70}
  Number of multi-qubit Operations: 133
Reduction in two-qubit gates compared to qiskit 24.432%!


global phase: 1.5723
           ┌────────┐  ┌────┐  ┌────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           ┌───┐    ┌──────────┐              ┌───┐   ┌────────────┐    ┌───┐     ┌───┐┌─────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
q_0 -> 0 ──┤ Rz(-π) ├──┤ √X ├──┤ Rz(-π) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├────┤ Rz(-π/2) ├──────────────┤ X ├───┤ Rz(1.5677) ├────┤ X ├─────┤ X ├┤ Rz(-3.1385) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [99]:
opt_transpiler_sim = noisy_sim(qc_opt, backend)

print("Own transpiler fidelity", round(hellinger_fidelity(own_transpiler_sim, reference_sim), 4))
print("Qiskit transpiler fidelity", round(hellinger_fidelity(qiskit_transpiler_sim, reference_sim), 4))
print("Own optimized transpiler fidelity", round(hellinger_fidelity(opt_transpiler_sim, reference_sim), 4))

Own transpiler fidelity 0.0482
Qiskit transpiler fidelity 0.9179
Own optimized transpiler fidelity 0.9141


In [ ]:
# Your work here! (optional!)


In [102]:
# grade your work!
num_qubits = 100
qc = QFT(num_qubits, do_swaps=False)
# Define a backend whose basis gate set and coupling map corresponds to a 133-qubit Heron quantum computer
target_backend = None

pm_student = StagedPassManager()
qc_student = pm_student.run(qc)
qc_qiskit = generate_preset_pass_manager(2, backend=target_backend).run(qc)
reduction_ratio = round(100-100*(get_qc_characteristics(qc_student)['num_multi_qubit_ops']/get_qc_characteristics(qc_qiskit)['num_multi_qubit_ops']), 3)

print("Reduction in two-qubit gates compared to qiskit {}%!".format(reduction_ratio))
print("N-qubit gates generated by Qiskit's default pass manager:", get_qc_characteristics(qc_qiskit)['num_multi_qubit_ops'])
print("N-qubit gates generated by your pass manager", get_qc_characteristics(qc_student)['num_multi_qubit_ops'])

Reduction in two-qubit gates compared to qiskit 99.98%!
N-qubit gates generated by Qiskit's default pass manager: 4950
N-qubit gates generated by your pass manager 1


In [251]:
from collections import defaultdict
from qiskit.circuit.library.standard_gates import RZGate # Import RZGate

class GatesPerQubit(AnalysisPass):

    """
      This analysis pass determines the number of single-qubit and two-qubit gates
      applied to each qubit in a circuit and stores the counts in the property_set.
      """

    def run(self, dag):
        one_q_op = defaultdict(int)  # Stores single-qubit gate count per qubit
        two_q_op = defaultdict(int)  # Stores two-qubit gate count per qubit

        for node in dag.op_nodes():
                qubit = node.qargs

                # Count the gates
                if len(node.qargs) == 1:
                      one_q_op[qubit[0]] += 1

                elif len(node.qargs) == 2:
                  for qubit in node.qargs:
                      two_q_op[qubit] += 1

                #print(two_q_op)
                #print(one_q_op)
                #print(dag.op_nodes())
                #print(dag.edges())
        self.property_set["one_q_op"] = dict(one_q_op)
        self.property_set["two_q_op"] = dict(two_q_op)
        return dag
    pass


In [252]:
grade_lab1_ex6(GatesPerQubit)


Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [253]:
qc = QFT(4, do_swaps=False)
qc = generate_preset_pass_manager(2, backend=backend).run(qc)
gpq = GatesPerQubit()
gpq(qc)
print("single-qubit gates on qubit", gpq.property_set["one_q_op"])
print("two-qubit gates on qubit", gpq.property_set["two_q_op"])
qc.draw(fold=-1, idle_wires=False)

single-qubit gates on qubit {Qubit(QuantumRegister(10, 'q'), 8): 3, Qubit(QuantumRegister(10, 'q'), 5): 4, Qubit(QuantumRegister(10, 'q'), 2): 6, Qubit(QuantumRegister(10, 'q'), 6): 8}
two-qubit gates on qubit {Qubit(QuantumRegister(10, 'q'), 8): 6, Qubit(QuantumRegister(10, 'q'), 5): 6, Qubit(QuantumRegister(10, 'q'), 2): 6, Qubit(QuantumRegister(10, 'q'), 6): 6}


global phase: 0
                                                                          ┌───┐┌──────────┐            ┌───┐┌─────────┐    ┌───┐    ┌──────────┐   ┌───┐    ┌──────────┐   ┌────┐   ┌──────────┐                                                              
q_1 -> 2 ─────────────────────────────────────────────────────────────────┤ X ├┤ Rz(-π/8) ├────────────┤ X ├┤ Rz(π/8) ├────┤ X ├────┤ Rz(-π/4) ├───┤ X ├────┤ Rz(3π/4) ├───┤ √X ├───┤ Rz(3π/4) ├─────────────■────────────────■───────────────────────────────
                                        ┌───┐┌──────────┐┌───┐┌──────────┐└─┬─┘└──┬────┬──┘┌──────────┐└─┬─┘└─────────┘    └─┬─┘    └──────────┘   └─┬─┘    └──────────┘   └────┘   └──────────┘             │                │                               
q_2 -> 5 ───────────────────────────────┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├──┼─────┤ √X ├───┤ Rz(7π/8) ├──┼───────────────────■───────────────────────■───────────■───────────────────────■───────────────────┼────────────────┼───────────────────────────────
                                        └─┬─┘└──────────┘└─┬─┘└──────────┘  │     └────┘   └──────────┘  │     ┌───┐   ┌───────────┐   ┌───┐    ┌──────────┐   ┌─┴─┐    ┌──────────┐   ┌─┴─┐    ┌─────────┐┌─┴─┐┌──────────┐┌─┴─┐┌──────────┐┌────┐┌─────────┐
q_0 -> 6 ─────────────────────────────────┼────────────────┼────────────────┼────────────────────────────┼─────┤ X ├───┤ Rz(-π/16) ├───┤ X ├────┤ Rz(π/16) ├───┤ X ├────┤ Rz(-π/8) ├───┤ X ├────┤ Rz(π/8) ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├┤ √X ├┤ Rz(π/2) ├
         ┌─────────┐┌────┐┌────────────┐  │                │                │                            │     └─┬─┘   └───────────┘   └─┬─┘    └──────────┘   └───┘    └──────────┘   └───┘    └─────────┘└───┘└──────────┘└───┘└──────────┘└────┘└─────────┘
q_3 -> 8 ┤ Rz(π/2) ├┤ √X ├┤ Rz(15π/16) ├──■────────────────■────────────────■────────────────────────────■───────■───────────────────────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         └─────────┘└────┘└────────────┘

In [254]:
pg = Gate('Peres', 3, params=[], label='PG')


In [255]:
qc_pg = QuantumCircuit(3)
qc_pg.append(pg, [0, 1, 2])
qc_pg.draw()

┌─────┐
q_0: ┤0    ├
     │     │
q_1: ┤1 PG ├
     │     │
q_2: ┤2    ├
     └─────┘

In [256]:
def get_qc_in(nq):
    # QFT circuit, feel free to use a previously defined pass manager for the QFT circuit
    qc_qft = QFT(nq, do_swaps=False)
    # part of the circuit including the Peres gate
    qc_inner = QuantumCircuit(nq)
    for i in range(1, nq-1):
        qc_inner.append(pg, [nq-i-2, nq-i-1, nq-1])

    qc_in = QuantumCircuit(nq)
    # add QFT circuit to qc_in
    qc_in.compose(qc_qft, range(nq), inplace=True)

    # perform swap gates
    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx( nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)

    qc_in.rz(pi, nq-1)
    # add circuit with peres gates
    qc_in.compose(qc_inner, range(nq), inplace=True)

    # perform swap gates
    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx( nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)
    # add inverse QFT circuit
    qc_in.compose(qc_qft.inverse(), range(nq), inplace=True)
    return qc_in

nq = 5
qc_in = get_qc_in(nq)
qc_in.draw(fold=-1)

┌──────┐          ┌───┐                                 ┌─────┐          ┌───┐               ┌───────┐
q_0: ┤0     ├──■───────┤ X ├───────■─────────────────────────┤0    ├──■───────┤ X ├───────■───────┤0      ├
     │      │  │       └─┬─┘┌───┐  │                  ┌─────┐│     │  │       └─┬─┘┌───┐  │       │       │
q_1: ┤1     ├──┼────■────┼──┤ X ├──┼──────■───────────┤0    ├┤1    ├──┼────■────┼──┤ X ├──┼────■──┤1      ├
     │      │  │    │    │  └─┬─┘  │      │    ┌─────┐│     ││     │  │    │    │  └─┬─┘  │    │  │       │
q_2: ┤2 QFT ├──┼────┼────┼────┼────┼──────┼────┤0    ├┤1    ├┤  PG ├──┼────┼────┼────┼────┼────┼──┤2 IQFT ├
     │      │  │  ┌─┴─┐  │    │    │    ┌─┴─┐  │     ││  PG ││     │  │  ┌─┴─┐  │    │    │  ┌─┴─┐│       │
q_3: ┤3     ├──┼──┤ X ├──┼────■────┼────┤ X ├──┤1 PG ├┤     ├┤     ├──┼──┤ X ├──┼────■────┼──┤ X ├┤3      ├
     │      │┌─┴─┐└───┘  │       ┌─┴─┐┌─┴───┴─┐│     ││     ││     │┌─┴─┐└───┘  │       ┌─┴─┐└───┘│       │
q_4: ┤4     ├┤ X ├───────■───────┤ X ├┤ Rz(π) ├┤2    ├┤2    ├┤2    ├┤ X ├───────■───────┤ X ├─────┤4      ├
     └──────┘└───┘               └───┘└───────┘└─────┘└─────┘└─────┘└───┘               └───┘     └───────┘

In [298]:
class PeresGateTranslation(TransformationPass):
    def get_peres_decomposition(self):
        qcsx = QuantumCircuit(2)
        qcsx.rz(pi / 4, 0)
        qcsx.rz(pi / 2, 1)
        qcsx.sx(1)
        qcsx.rz(pi / 2, 1)
        qcsx.cx(0, 1)
        qcsx.rz(-pi / 4, 1)
        qcsx.cx(0, 1)
        qcsx.rz(3 * pi / 4, 1)
        qcsx.sx(1)
        qcsx.rz(pi / 2, 1)

        qcsx_inv = QuantumCircuit(2)
        qcsx_inv.rz(pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(-pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 2, 0)
        qcsx_inv.rz(pi / 2, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 2, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.sx(1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(-pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 4, 0)

        qc_dec = QuantumCircuit(3)
        qc_dec.cx(0, 1)
        qc_dec.cx(1, 0)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.cx(1, 0)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx_inv, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.cx(0, 1)
        return qc_dec
    def run(self,dag):
        decomposed = False
        for node in dag.op_nodes():
            #print(node.name)
            if node.op.name == 'Peres':
             #   print(node.name)
                replacement_circuit = self.get_peres_decomposition()
                replacement_dag = circuit_to_dag(replacement_circuit)
                dag.substitute_node_with_dag(node, replacement_dag)
                decomposed = True
            #else:
             #     print("not peres")
        return dag
qc_in.draw()

┌──────┐          ┌───┐                                 ┌─────┐          »
q_0: ┤0     ├──■───────┤ X ├───────■─────────────────────────┤0    ├──■───────»
     │      │  │       └─┬─┘┌───┐  │                  ┌─────┐│     │  │       »
q_1: ┤1     ├──┼────■────┼──┤ X ├──┼──────■───────────┤0    ├┤1    ├──┼────■──»
     │      │  │    │    │  └─┬─┘  │      │    ┌─────┐│     ││     │  │    │  »
q_2: ┤2 QFT ├──┼────┼────┼────┼────┼──────┼────┤0    ├┤1    ├┤  PG ├──┼────┼──»
     │      │  │  ┌─┴─┐  │    │    │    ┌─┴─┐  │     ││  PG ││     │  │  ┌─┴─┐»
q_3: ┤3     ├──┼──┤ X ├──┼────■────┼────┤ X ├──┤1 PG ├┤     ├┤     ├──┼──┤ X ├»
     │      │┌─┴─┐└───┘  │       ┌─┴─┐┌─┴───┴─┐│     ││     ││     │┌─┴─┐└───┘»
q_4: ┤4     ├┤ X ├───────■───────┤ X ├┤ Rz(π) ├┤2    ├┤2    ├┤2    ├┤ X ├─────»
     └──────┘└───┘               └───┘└───────┘└─────┘└─────┘└─────┘└───┘     »
«     ┌───┐               ┌───────┐
«q_0: ┤ X ├───────■───────┤0      ├
«     └─┬─┘┌───┐  │       │       │
«q_1: ──┼──┤ X ├──┼────■──┤1      ├
«       │  └─┬─┘  │    │  │       │
«q_2: ──┼────┼────┼────┼──┤2 IQFT ├
«       │    │    │  ┌─┴─┐│       │
«q_3: ──┼────■────┼──┤ X ├┤3      ├
«       │       ┌─┴─┐└───┘│       │
«q_4: ──■───────┤ X ├─────┤4      ├
«               └───┘     └───────┘

In [299]:
grade_lab1_ex7(PeresGateTranslation, pg)


Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [ ]:
# Your code here! (Optional!)
"""As a last optional exercise, reduce the overhead of transpiling qc_in (the quantum circuit from exercise 3b) for the target quantum computer defined
 in the last optional exercise with any of the approaches you have been made familiar with during this exercise!"""


In [ ]:
import datetime
from IPython.display import HTML, display


def qiskit_copyright(line="", cell=None):
    """IBM copyright"""
    now = datetime.datetime.now()

    html = "<div style='width: 100%; background-color:#d5d9e0;"
    html += "padding-left: 10px; padding-bottom: 10px; padding-right: 10px; padding-top: 5px'>"
    html += "<p>&copy; Copyright IBM 2017, %s.</p>" % now.year
    html += "<p>This code is licensed under the Apache License, Version 2.0. You may<br>"
    html += "obtain a copy of this license in the LICENSE.txt file in the root directory<br> "
    html += "of this source tree or at http://www.apache.org/licenses/LICENSE-2.0."

    html += "<p>Any modifications or derivative works of this code must retain this<br>"
    html += "copyright notice, and modified files need to carry a notice indicating<br>"
    html += "that they have been altered from the originals.</p>"
    html += "</div>"
    return display(HTML(html))


qiskit_copyright()